# The Importance of Cleaning the Text

After a few different iterations, I think that I have found a pretty good way to clean the questions to improve the performance of a model. I was able to reduce my loss value by a few points because of this method.  Feel free to use this code and improve upon the method!

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from string import punctuation

In [5]:
train = pd.read_csv("../train.csv")#[:10000]
test = pd.read_csv("../test.csv")#[:10000]

In [6]:
train.shape

(404290, 6)

In [3]:
# Check for any null values
print(train.isnull().sum())
print(test.isnull().sum())

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64
test_id      0
question1    0
question2    0
dtype: int64


In [4]:
# Add the string 'empty' to empty strings
train = train.fillna('empty')
test = test.fillna('empty')

In [5]:
# Preview some of the pairs of questions
a = 0 
for i in range(a,a+10):
    print(train.question1[i])
    print(train.question2[i])
    print()

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?
()
What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
()
How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?
()
Why am I mentally very lonely? How can I solve it?
Find the remainder when [math]23^{24}[/math] is divided by 24,23?
()
Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?
Which fish would survive in salt water?
()
Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?
I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?
()
Should I buy tiago?
What keeps childern active and far from phone and video games?
()
How can I be a good geologist?
What should I do to be a great geologist?
()
When do 

In [6]:
stop_words = ['the','a','an','and','but','if','or','because','as','what','which','this','that','these','those','then',
              'just','so','than','such','both','through','about','for','is','of','while','during','to','What','Which',
              'Is','If','While','This']

In [24]:
def text_to_wordlist(text, remove_stop_words=False, stem_words=False):
    # Clean the text, with the option to remove stop_words and to stem words.

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"what's", "what", text)
    text = re.sub(r"What's", "What", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"I'm", "I am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " America ", text)
    text = re.sub(r" USA ", " America ", text)
    text = re.sub(r" u s ", " America ", text)
    text = re.sub(r" uk ", " England ", text)
    text = re.sub(r" UK ", " England ", text)
    text = re.sub(r"india", "India", text)
    text = re.sub(r"switzerland", "Switzerland", text)
    text = re.sub(r"china", "China", text)
    text = re.sub(r"chinese", "Chinese", text) 
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r"quora", "Quora", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r"KMs", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text) 
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iPhone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"gps", "GPS", text)
    text = re.sub(r"gst", "GST", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"dna", "DNA", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the US", "America", text)
    text = re.sub(r"Astrology", "astrology", text)
    text = re.sub(r"Method", "method", text)
    text = re.sub(r"Find", "find", text) 
    text = re.sub(r"banglore", "Banglore", text)
    text = re.sub(r" J K ", " JK ", text)
    
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [25]:
def process_questions(question_list, questions, question_list_name, dataframe):
    '''transform questions and display progress'''
    for question in questions:
        question_list.append(text_to_wordlist(question))
        if len(question_list) % 100000 == 0:
            progress = len(question_list)/len(dataframe) * 100
            print("{} is {}% complete.".format(question_list_name, round(progress, 1)))

In [26]:
train_question1 = []
process_questions(train_question1, train.question1, 'train_question1', train)

In [27]:
train_question2 = []
process_questions(train_question2, train.question2, 'train_question2', train)

In [28]:
test_question1 = []
process_questions(test_question1, test.question1, 'test_question1', test)

In [29]:
test_question2 = []
process_questions(test_question2, test.question2, 'test_question2', test)

In [30]:
# Preview some transformed pairs of questions
a = 0 
for i in range(a,a+10):
    print(train_question1[i])
    print(train_question2[i])
    print()

What is the step by step guide to invest in share market in India 
What is the step by step guide to invest in share market 
()
What is the story of Kohinoor Koh i Noor Diamond 
What would happen if the Indian government stole the Kohinoor Koh i Noor diamond back 
()
How can I increase the speed of my internet connection while using a VPN 
How can Internet speed be increased by hacking through DNS 
()
Why am I mentally very lonely How can I solve it 
find the remainder when math 23 24 math is divided by 24 23 
()
Which one dissolve in water quickly sugar salt methane and carbon di oxide 
Which fish would survive in salt water 
()
astrology I am a Capricorn Sun Cap moon and cap rising what does that say about me 
I am a triple Capricorn Sun Moon and ascendant in Capricorn What does this say about me 
()
Should I buy tiago 
What keeps childern active and far from phone and video games 
()
How can I be a good geologist 
What should I do to be a great geologist 
()
When do you use instead 

In [31]:
len(train_question1)

10000

In [34]:
pickle.dump([list(train_question1) + list(train_question2)], open("sentences_train_10000.pkl", "w"))

In [33]:
train_question1

['What is the step by step guide to invest in share market in India ',
 'What is the story of Kohinoor Koh i Noor Diamond ',
 'How can I increase the speed of my internet connection while using a VPN ',
 'Why am I mentally very lonely How can I solve it ',
 'Which one dissolve in water quickly sugar salt methane and carbon di oxide ',
 'astrology I am a Capricorn Sun Cap moon and cap rising what does that say about me ',
 'Should I buy tiago ',
 'How can I be a good geologist ',
 'When do you use instead of ',
 'Motorola company Can I hack my Charter Motorolla DCX3400 ',
 'method to find separation of slits using fresnel biprism ',
 'How do I read and find my YouTube comments ',
 'What can make Physics easy to learn ',
 'What was your first sexual experience like ',
 'What are the laws to change your status from a student visa to a green card in America how do they compare to the immigration laws in Canada ',
 'What would a Trump presidency mean for current international master s stude

In [14]:
import pickle
s = pickle.load(open("sentences_train_10000.pkl"))

In [15]:
predPair = pickle.load(open("predPair.pkl"))

In [16]:
len(predPair)

10000

In [17]:
from sklearn.metrics import classification_report
print classification_report(list(train["is_duplicate"]), predPair)

             precision    recall  f1-score   support

          0       0.79      0.34      0.48      6289
          1       0.43      0.84      0.57      3711

avg / total       0.65      0.53      0.51     10000



In [18]:
predPair_5 = pickle.load(open("predPair_5.pkl"))
predPair_5_dup = map(lambda x: x[2], predPair_5)

In [19]:
from sklearn.metrics import classification_report
y_pred = predPair_5_dup
y_true = list(train["is_duplicate"])
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.73      0.42      0.53      6289
          1       0.43      0.74      0.54      3711

avg / total       0.62      0.54      0.54     10000



In [20]:
predPair_1 = pickle.load(open("predPair_1.pkl"))
predPair_1_dup = map(lambda x: x[2], predPair_1)

In [21]:
from sklearn.metrics import classification_report
y_pred = predPair_1_dup
y_true = list(train["is_duplicate"])
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.69      0.57      0.62      6289
          1       0.44      0.56      0.49      3711

avg / total       0.60      0.57      0.58     10000



In [22]:
cnt_0_0 = 0
cnt_0_1 = 0
cnt_1_0 = 0
cnt_1_1 = 0
for i in range(0,10000):
    if y_true[i] == 1 and y_pred[i] == 1:
        cnt_1_1 +=1
    if y_true[i] == 1 and y_pred[i] == 0:
        cnt_1_0 +=1
    if y_true[i] == 0 and y_pred[i] == 1:
        cnt_0_1 +=1
    if y_true[i] == 0 and y_pred[i] == 0:
        cnt_0_0 +=1

In [23]:
np.array([[cnt_0_0, cnt_0_1], [cnt_1_0, cnt_1_1]])

array([[3589, 2700],
       [1620, 2091]])

In [24]:
from collections import Counter
Counter(y_true)

Counter({0: 6289, 1: 3711})

In [25]:
sDist = pickle.load(open("sDist.pkl"))
sDist = map(lambda x: x[0][0], sDist)

In [26]:
import numpy as np
np.corrcoef(sDist, y_true)

array([[ 1.        ,  0.36723104],
       [ 0.36723104,  1.        ]])

In [27]:
sDist = np.array(sDist)
sDist_prob = ((sDist - (-1)*(1 - 0))/(1 - (-1))) + 0

In [28]:
sDist_prob

array([ 0.99640392,  0.91565851,  0.88714205, ...,  0.75219096,
        0.92136174,  0.91641575])

In [29]:
import numpy as np
from sklearn import metrics
y = np.array(y_true)
pred = np.array(sDist_prob)
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.72315383106157016

In [30]:
d = list(sDist_prob)
d_not = list(1 - sDist_prob)
probs = zip(d, d_not)
probs = list(probs)

In [31]:
from sklearn.metrics import log_loss
log_loss(y_true, probs)

1.2906591999309924

In [32]:
mu = np.mean(sDist)

In [55]:
mu

0.67831062440222667

In [33]:
y_pred  = (sDist >= mu) + 0

In [34]:
print classification_report(y_true, y_pred)

             precision    recall  f1-score   support

          0       0.82      0.51      0.63      6289
          1       0.50      0.81      0.62      3711

avg / total       0.70      0.62      0.63     10000



In [35]:
train["pred"] = y_pred

In [53]:
train["prob"] = sDist

In [54]:
train.to_csv("train_analysis.csv")

In [44]:
analysis = train.loc[train["is_duplicate"]!=y_pred, :].reset_index()
analysis

,index,id,qid1,qid2,question1,question2,is_duplicate,pred
0,0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1
1,1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,1
2,2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1
3,8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,1
4,9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,1
5,14,14,29,30,What are the laws to change your status from a...,What are the laws to change your status from a...,0,1
6,15,15,31,32,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,1,0
7,17,17,35,36,Why do girls want to be friends with the guy t...,How do guys feel after rejecting a girl?,0,1
8,19,19,39,40,Which is the best digital marketing institutio...,Which is the best digital marketing institute ...,0,1
9,21,21,43,44,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,0,1


In [51]:
for i in range(0,1000):
    print analysis["question1"][i]
    print analysis["question2"][i]
    print "true: ", analysis["is_duplicate"][i], "pred: ", analysis["pred"][i]
    print "\n"

What is the step by step guide to invest in share market in india?
What is the step by step guide to invest in share market?
true:  0 pred:  1


What is the story of Kohinoor (Koh-i-Noor) Diamond?
What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?
true:  0 pred:  1


How can I increase the speed of my internet connection while using a VPN?
How can Internet speed be increased by hacking through DNS?
true:  0 pred:  1


When do you use シ instead of し?
When do you use "&" instead of "and"?
true:  0 pred:  1


Motorola (company): Can I hack my Charter Motorolla DCX3400?
How do I hack Motorola DCX3400 for free internet?
true:  0 pred:  1


What are the laws to change your status from a student visa to a green card in the US, how do they compare to the immigration laws in Canada?
What are the laws to change your status from a student visa to a green card in the US? How do they compare to the immigration laws in Japan?
true:  0 pred:  1


What would a Tru

In [ ]:
train["prob"] = 

In [52]:
train.to_csv("analysis.csv")

In [38]:
train.loc[train["is_duplicate"]==y_pred, :]

,id,qid1,qid2,question1,question2,is_duplicate,pred
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,1
10,10,21,22,Method to find separation of slits using fresn...,What are some of the things technicians can te...,0,0
11,11,23,24,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1,1
12,12,25,26,What can make Physics easy to learn?,How can you make physics easy to learn?,1,1
13,13,27,28,What was your first sexual experience like?,What was your first sexual experience?,1,1
16,16,33,34,What does manipulation mean?,What does manipulation means?,1,1


In [102]:
cnt_0_0 = 0
cnt_0_1 = 0
cnt_1_0 = 0
cnt_1_1 = 0
for i in range(0,10000):
    if y_true[i] == 1 and y_pred[i] == 1:
        cnt_1_1 +=1
    if y_true[i] == 1 and y_pred[i] == 0:
        cnt_1_0 +=1
    if y_true[i] == 0 and y_pred[i] == 1:
        cnt_0_1 +=1
    if y_true[i] == 0 and y_pred[i] == 0:
        cnt_0_0 +=1

In [103]:
np.array([[cnt_0_0, cnt_0_1], [cnt_1_0, cnt_1_1]])

array([[3229, 3060],
       [ 699, 3012]])

## We reduced 1 (duplicates) being missclassified as 0 (not duplicates)
## We are pretty good at detecting 1's (duplicates) 
## lot of non-duplicates are classified as duplicates because we are not getting rid of missclassifications

In [106]:
699./3012.

0.23207171314741035

### What we are concerned as a user is we should not miss out on duplicates recall of duplicates(1) should be high. Then user can make a choice... We dont wanna miss out on opportunity cost of gettting a good answer on a duplicate question. But then we also have limited window size

In [116]:
y_pred_dummy = np.zeros((10000,))
y_pred_dummy = list(y_pred_dummy)

In [117]:
from sklearn.metrics import classification_report
print classification_report(y_true, y_pred_dummy)

             precision    recall  f1-score   support

          0       0.63      1.00      0.77      6289
          1       0.00      0.00      0.00      3711

avg / total       0.40      0.63      0.49     10000



/Users/Taran/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [118]:
y_pred_dummy = np.ones((10000,))
y_pred_dummy = list(y_pred_dummy)
from sklearn.metrics import classification_report
print classification_report(y_true, y_pred_dummy)

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      6289
          1       0.37      1.00      0.54      3711

avg / total       0.14      0.37      0.20     10000

